In [1]:
from model import *
from data import *

Using TensorFlow backend.


## Train your Unet with medical data
it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Set hyperparameters

* epochs_no - more runs and improvs generalisation of the model
* batch_size - number of samples to run at each epoch step

In [2]:
# Hyper parametrs
batch_size = 8
epochs_no = 2000
steps_per_epoch = epochs_no / batch_size
epochs_no = 200

### Model Definition

In [3]:
input_size = (256,256,1) # actual data
#input_size = (256,256,1) # Augmented data
model = unet(input_size=input_size)
# Monitor can be either acc or val_loss or val_acc
model_checkpoint = ModelCheckpoint('unet_halfbody.hdf5', monitor='loss',verbose=1, save_best_only=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

D:\Research\ML\unet_ml\model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


Stack Over Flow discussion - https://stackoverflow.com/questions/47930176/printing-out-the-validation-accuracy-to-the-console-for-every-batch-or-epoch-ke

### Train with data generator

In [4]:
#if you don't want to do data augmentation, set data_gen_args as an empty dict.
#data_gen_args = dict()

data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

In [7]:
# Actual data size ~train, label folders and uses augmented saved data in ~aug folder
trainGene = trainGenerator(batch_size,'data/halfbody/train','image','label',data_gen_args,save_to_dir = None)

# Downsampled data and no augmentation
#myGene = trainGenerator(batch_size,'data/halfbody/train','image_downsampled','label_downsampled',data_gen_args,save_to_dir = None)

In [5]:
#
testGene = trainGenerator(batch_size,'data/halfbody/train','test_image','test_label',data_gen_args,save_to_dir = None)

#### Model Fit Generator

* https://keras.io/models/model/
* https://www.pyimagesearch.com/2018/12/24/how-to-use-keras-fit-and-fit_generator-a-hands-on-tutorial/

In [ ]:
HistDict = model.fit_generator(trainGene,steps_per_epoch=steps_per_epoch,epochs=epochs_no, verbose=1, validation_data = testGene, validation_steps = steps_per_epoch,callbacks=[model_checkpoint])

Epoch 1/200
Found 770 images belonging to 1 classes.
Found 1259 images belonging to 1 classes.
Found 770 images belonging to 1 classes.
Found 1259 images belonging to 1 classes.
250/250 [==============================] - 274s 1s/step - loss: 0.3483 - acc: 0.9772 - val_loss: 0.2750 - val_acc: 0.9829

Epoch 00001: loss improved from inf to 0.34846, saving model to unet_halfbody.hdf5
Epoch 2/200
250/250 [==============================] - 260s 1s/step - loss: 0.3353 - acc: 0.9792 - val_loss: 0.2746 - val_acc: 0.9830

Epoch 00002: loss improved from 0.34846 to 0.33565, saving model to unet_halfbody.hdf5
Epoch 3/200
250/250 [==============================] - 260s 1s/step - loss: 0.3403 - acc: 0.9789 - val_loss: 0.2731 - val_acc: 0.9831

Epoch 00003: loss did not improve from 0.33565
Epoch 4/200
250/250 [==============================] - 259s 1s/step - loss: 0.3374 - acc: 0.9791 - val_loss: 0.2748 - val_acc: 0.9830

Epoch 00004: loss did not improve from 0.33565
Epoch 5/200
250/250 [=========

### plot the training loss and accuracy

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
N = epochs_no
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), HistDict.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), HistDict.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), HistDict.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), HistDict.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="best")
plt.savefig("data/halfbody/plot_halfbody.png")

### Train with npy file

### a. Train with Augmented data

In [ ]:
imgs_train,imgs_mask_train = geneTrainNpy("data/halfbody/train/aug/","data/halfbody/train/aug/")
#HistDict = model.fit(imgs_train, imgs_mask_train, batch_size=batch_sizes, epochs=epochs_no, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### b. Train with Original data

In [ ]:
imgs_train,imgs_mask_train = geneTrainNpy("data/halfbody/train/image_downsampled/","data/halfbody/train/label_downsampled/", image_prefix="",mask_prefix="")
model.fit(imgs_train, imgs_mask_train, batch_size=batch_size, epochs=epochs_no, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### Test your model and save predicted results

Link to Stack overflow discussion - https://stackoverflow.com/questions/52946110/u-net-low-contrast-test-images-predict-output-is-grey-box

### Set the output data dimension here

input and target size must match!

In [ ]:
## Resample to original sizes
input_size = (512,512,1)
target_size=(512,512)

### Test model on n datasets

In [ ]:
import os
test_path = 'data/halfbody/test_orig'
num_images = 0
for entry in os.listdir(test_path):
    if os.path.isfile(os.path.join(test_path, entry)):
        if entry.lower().endswith(('.png', '.jpg', '.jpeg')):
            num_images = num_images + 1

In [ ]:
# Path toread datasets tp apply prediction to
testGene = testGenerator(test_path, num_image=num_images, target_size=target_size)

In [ ]:
model = unet(input_size=input_size)
model.load_weights("unet_halfbody.hdf5")
results = model.predict_generator(testGene,num_images,verbose=1)

### Save the predicted data

In [ ]:
# Input is image and path to save the predicted datasets to
saveResult("data/halfbody/test_orig_label",results)